In [7]:
import gradio as gr
import os
import tempfile
import numpy as np
import soundfile as sf
from dotenv import load_dotenv
from openai import OpenAI
import random

load_dotenv()
client = OpenAI()

languages = [
    'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 
    'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chichewa', 'Chinese', 'Corsican', 
    'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Filipino', 'Finnish', 
    'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 
    'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 
    'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Kinyarwanda', 
    'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 
    'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 
    'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Odia', 'Pashto', 'Persian', 'Polish', 'Portuguese', 
    'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 
    'Sindhi', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 
    'Swedish', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Turkish', 'Turkmen', 'Ukrainian', 
    'Urdu', 'Uyghur', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu'
]


def translate_word(word, source_lang="da", target_lang="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a translator from {source_lang} to {target_lang}. Your output should be similar to a dictionary definition."},
            {"role": "user", "content": f"Translate the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

def generate_sentence(word, language="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a sentence generator in {language}."},
            {"role": "user", "content": f"Generate a simple sentence using the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

def add_silence(input_file, output_file, silence_duration_sec=0.5):
    data, samplerate = sf.read(input_file)
    silence_samples = int(silence_duration_sec * samplerate)
    silence = np.zeros((silence_samples, data.shape[1])) if data.ndim > 1 else np.zeros(silence_samples)
    new_data = np.concatenate((silence, data), axis=0)
    sf.write(output_file, new_data, samplerate)

def get_audio(text, lang="da"):
    voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
    selected_voice = random.choice(voices)
    prompt = f"Følgende tekst er dansk: '{text}'"

    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        temp_audio_path = temp_audio_file.name
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=selected_voice,
            input=prompt,
        ) as response:
            response.stream_to_file(temp_audio_path)

    output_file_path = f"audio_{text}.mp3"
    add_silence(temp_audio_path, output_file_path, silence_duration_sec=1)
    os.remove(temp_audio_path)
    return output_file_path

def save_into_database(word, translation, sentence, audio_path):
    # Implement your database saving logic here
    print(f"Saving to database: {word}, {translation}, {sentence}, {audio_path}")
    return "Data saved successfully"

def process_word(danish_word, custom_sentence="", language=""):
    translation = translate_word(danish_word, language)
    sentence = custom_sentence if custom_sentence else generate_sentence(translation, language)
    audio_path = get_audio(danish_word, language)
    
    # Save to database
    save_result = save_into_database(danish_word, translation, sentence, audio_path)
    
    return translation, sentence, audio_path, save_result

# Gradio interface
iface = gr.Interface(
    fn=process_word,
    inputs=[
        gr.Textbox(label="Danish Word"),
        gr.Textbox(label="Custom Sentence (optional)"),
        gr.Dropdown(
            choices = languages, 
            value="Danish", 
            label="Language", 
        ),
    ],
    outputs=[
        gr.Textbox(label="English Translation"),
        gr.Textbox(label="Generated/Custom Sentence"),
        gr.Audio(label="Pronunciation"),
        gr.Textbox(label="Database Save Result")
    ],
    title="Danish-English Language Learning App",
    description="Enter a Danish word to get its English translation, a sample sentence, and pronunciation."
)

if __name__ == "__main__":
    iface.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Saving to database: kæreste, noun: kæreste (plural: kærester)

1. (noun) boyfriend/girlfriend; partner in a romantic relationship

Example: "Han er min kæreste" translates to "He is my boyfriend" in English., "De er mine kærester" betyder "They are my partners" på engelsk., audio_kæreste.mp3
